In [13]:
%matplotlib inline
import pennylane as qml
from pennylane import numpy as np
from dimod import BQM, ExactSolver
import pandas as pd
from IPython.display import display, Math

![problem graph](./assets/graph.png)

In [3]:
idx = pd.Index(['A', 'B', 'C', 'D', 'E'])
df = pd.DataFrame([
    np.array([0, 5.8, 5.8, 7.6, 8.8], dtype=np.float16),
    np.array([5.8, 0, 2.4, 4.3, 4.3], dtype=np.float16),
    np.array([5.8, 2.4, 0, 3.5, 4.7], dtype=np.float16),
    np.array([7.6, 4.3, 3.5, 0, 1.4], dtype=np.float16),
    np.array([8.8, 4.3, 4.7, 1.4, 0], dtype=np.float16),
    ], 
    columns=['A', 'B', 'C', 'D', 'E'], 
    index=idx)
df_cap = df.style.set_caption('Points Distances')
df_cap

,A,B,C,D,E
A,0.0,5.8,5.8,7.6,8.8
B,5.8,0.0,2.4,4.3,4.3
C,5.8,2.4,0.0,3.5,4.7
D,7.6,4.3,3.5,0.0,1.4
E,8.8,4.3,4.7,1.4,0.0


---

In [10]:
# testing using only the first step

P = df['A'].sum()

quadratic = {}

for i,v in enumerate(df['A'][1:]):
    print(v)
    quadratic[('x0', f'x{i+1}')] = v
    
qubo = BQM(quadratic, "BINARY")
qubo.add_linear_equality_constraint([(f'x{i}', 1) for i in range(5)], P, -1)
#qubo.to_ising()
print(qubo)

5.8
5.8
7.6
8.8
BinaryQuadraticModel({'x0': -28.0, 'x1': -28.0, 'x2': -28.0, 'x3': -28.0, 'x4': -28.0}, {('x1', 'x0'): 61.80078125, ('x2', 'x0'): 61.80078125, ('x2', 'x1'): 56.0, ('x3', 'x0'): 63.6015625, ('x3', 'x1'): 56.0, ('x3', 'x2'): 56.0, ('x4', 'x0'): 64.796875, ('x4', 'x1'): 56.0, ('x4', 'x2'): 56.0, ('x4', 'x3'): 56.0}, 28.0, 'BINARY')


In [5]:
sampler = ExactSolver()
sampleset = sampler.sample(qubo)
print(sampleset)

   x0 x1 x2 x3 x4     energy num_oc.
1   1  0  0  0  0        0.0       1
3   0  1  0  0  0        0.0       1
7   0  0  1  0  0        0.0       1
15  0  0  0  1  0        0.0       1
31  0  0  0  0  1        0.0       1
0   0  0  0  0  0       28.0       1
4   0  1  1  0  0       28.0       1
8   0  0  1  1  0       28.0       1
12  0  1  0  1  0       28.0       1
16  0  0  0  1  1       28.0       1
24  0  0  1  0  1       28.0       1
28  0  1  0  0  1       28.0       1
2   1  1  0  0  0  33.800781       1
6   1  0  1  0  0  33.800781       1
14  1  0  0  1  0  35.601562       1
30  1  0  0  0  1  36.796875       1
11  0  1  1  1  0      112.0       1
19  0  1  0  1  1      112.0       1
23  0  0  1  1  1      112.0       1
27  0  1  1  0  1      112.0       1
5   1  1  1  0  0 123.601562       1
9   1  0  1  1  0 125.402344       1
13  1  1  0  1  0 125.402344       1
25  1  0  1  0  1 126.597656       1
29  1  1  0  0  1 126.597656       1
17  1  0  0  1  1 128.398438       1
2

In [6]:
result = sampleset.to_pandas_dataframe()
result = result.drop(['num_occurrences'], axis=1)
result.head()

,x0,x1,x2,x3,x4,energy
0,0,0,0,0,0,28.000000
1,1,0,0,0,0,0.000000
2,1,1,0,0,0,33.800781
3,0,1,0,0,0,0.000000
4,0,1,1,0,0,28.000000


In [7]:
min_energy = result['energy'].min()
print('min energy ExactSolver(): ', min_energy)

min energy ExactSolver():  0.0


In [8]:
variables = sampleset.variables.to_serializable()
bit_strings = result.where(result['energy'] == min_energy).dropna()[variables].astype(int).astype(str).agg(''.join, axis=0).tolist()
bit_strings

['10000', '01000', '00100', '00010', '00001']

---

In [32]:
letters = ('A', 'B', 'C', 'D', 'E')

variables = ''
using_letters = ''

for i in range(5):
    for j in range(5):
        if(i == j):
            continue

        using_letters += f'{i}{letters[j]}'
        variables += f'x{i}x{j} '

        if(not (i == 4 and j == 3)):
            variables += '+'
            using_letters += '+'

display(Math(variables))
display(Math(using_letters))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [33]:
letters = ('A', 'B', 'C', 'D', 'E')

variables = ''
using_letters = ''

for i in range(5):
    for j in range(5):
        for k in range(5):
            if(i == j or j == k or k == i):
                continue

            #timestamp - last step - actual step
            
            using_letters += f'{i}{letters[j]}{letters[k]}'
            variables += f'x{i}{j}x{k} '
    
            if(not (i == 4 and j == 4 and k == 3)):
                variables += '+'
                using_letters += '+'

display(Math(variables))
display(Math(using_letters))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

---

In [39]:
n_steps = df.shape[0]

In [35]:
total_points = df.shape[0]
P = 100

quadratic = {}


for timestep in range(n_steps):
    for i in range(total_points):        
        first_variable = f'x{timestep}{i}'

        for j in range(total_points):
            

        if(timestep == 0):
            break
        print(first_variable)
    
#qubo = BQM(quadratic, "BINARY")
#qubo.add_linear_equality_constraint([(f'x{i}', 1) for i in range(5)], P, -1)
#qubo.to_ising()
#print(qubo)

x00
x01
x02
x03
x04
x10
x11
x12
x13
x14
x20
x21
x22
x23
x24
x30
x31
x32
x33
x34
x40
x41
x42
x43
x44


In [9]:
device = qml.device("default.qubit", wires=H.wires)

NameError: name 'H' is not defined

In [20]:
@qml.qnode(device)
def optimize_circuit(params):
    for param, wire in zip(params, H.wires):
        qml.RY(param, wires=wire)
    return qml.expval(H)

In [21]:
params = np.random.rand(len(H.wires))
optimizer = qml.AdagradOptimizer(stepsize=0.1)
epochs = 200

for epoch in range(epochs):
    params = optimizer.step(optimize_circuit, params)
    print(f"[*] epoch={epoch} : {params}")
params

[*] epoch=0 : [0.70489607 0.88236221 1.00367361 0.48573707 1.04853627]
[*] epoch=1 : [0.77652737 0.95357536 1.07417711 0.56116959 1.118807  ]
[*] epoch=2 : [0.83436954 1.01115725 1.13087402 0.6238906  1.17521255]
[*] epoch=3 : [0.88344513 1.06024285 1.17902257 0.67828707 1.22305136]
[*] epoch=4 : [0.92621328 1.10332502 1.22115928 0.72660861 1.26487441]
[*] epoch=5 : [0.96410487 1.14184831 1.25874731 0.77021066 1.30215113]
[*] epoch=6 : [0.99804411 1.17674509 1.29272769 0.80999432 1.33582554]
[*] epoch=7 : [1.02867143 1.20866299 1.32375211 0.84660016 1.36655083]
[*] epoch=8 : [1.05645367 1.2380766  1.35229687 0.88050643 1.39480398]
[*] epoch=9 : [1.08174421 1.26534819 1.37872472 0.91208388 1.42094804]
[*] epoch=10 : [1.10481855 1.29076347 1.40332113 0.94162873 1.44526845]
[*] epoch=11 : [1.12589653 1.31455378 1.42631677 0.96938353 1.46799577]
[*] epoch=12 : [1.14515679 1.33691065 1.44790224 0.99555105 1.48932034]
[*] epoch=13 : [1.16274674 1.35799564 1.468238   1.02030384 1.50940231]
[*

tensor([0.0130467 , 3.0567714 , 3.07132476, 3.01928098, 3.07669021], requires_grad=True)

In [22]:
device2 = qml.device("default.qubit", wires=H.wires, shots=1)

In [23]:
@qml.qnode(device2)
def eval_circuit(params):
    for param, wire in zip(params, H.wires):
        qml.RY(param, wires=wire)
    return qml.sample()

In [24]:
eval_circuit(params)

array([0, 1, 1, 1, 1])